### naver_stock

In [1]:
import pandas as pd

word_scores = {}  # 키워드별 등락률 스코어
word_counts = {}  # 키워드 등장 횟수

naver_stock = pd.read_csv('naver_stock.csv', encoding='utf-8')
naver_stock.head()

,Date,Close,Diff,Open,High,Low,Volume,Nasdaq_Close,Nasdaq_change,Label
0,2022.05.31,288000,7000,281500,289500,277500,1511249,12131.13,0.0333,1
1,2022.05.30,281000,11000,274000,282500,273500,751170,12131.13,0.0333,1
2,2022.05.27,270000,4000,271000,272000,268500,481191,11740.65,0.0268,0
3,2022.05.26,266000,0,269500,272500,266000,541194,11434.74,0.0151,0
4,2022.05.25,266000,0,266500,268500,262500,533307,11264.45,-0.0235,0


In [2]:
len(naver_stock)

273

In [3]:
date2idx = {}

for i in range(len(naver_stock)):
    stock_dates = naver_stock.iloc[i]['Date'].split('.')
    stock_dates[1] = stock_dates[1].lstrip('0')
    stock_date = " ".join(stock_dates)    
    date2idx[stock_date] = i
    
print(len(date2idx))

273


In [4]:
import pickle

with open('date2idx.pickle', 'wb') as f:
    pickle.dump(date2idx, f)

### naver_article

In [5]:
naver_article = pd.read_csv('naver.csv', encoding='utf-8')
naver_article.head()

,Title,Article,Date,Newspaper,Link
0,"[단독] 네이버 ""빅테크 전망 밝지 않지만…우린 다를 것""",골드만삭스 투자자행사서김남선 CFO 보수적인 예측새 광고상품으로 위기 돌파거시적 전...,2022.5.31 17:33,매일경제,https://finance.naver.com/item/news_read.naver...
1,"주가 상승세 탄 네이버, 29만원 고지 탈환할까 [증시이슈]",증권가 하반기 실적 기대네이버 주가가 상승 마감했다연합뉴스네이버 주가가 상승 마감했...,2022.5.31 16:54,이코노미스트,https://finance.naver.com/item/news_read.naver...
2,(영상)남들 철수하는데 새벽배송 도전장 내민 네이버...왜,네이버 하반기 새벽배송 서비스 계획CJ대한통운 물류 인프라 시너지 노려31일 이데일...,2022.5.31 16:27,이데일리,https://finance.naver.com/item/news_read.naver...
3,투표 전 네이버 자격증으로 신분 확인 가능,네이버 앱에서 발급받은 모바일 자격증 활용 가능 파이낸셜뉴스 네이버는 오는 6월 1...,2022.5.31 14:25,파이낸셜뉴스,https://finance.naver.com/item/news_read.naver...
4,"""경기침체에도 웹툰은 본다…네이버 목표주가 38만원""",사진네이버웹툰미래에셋증권은 31일 네이버에 대해 콘텐츠 사업의 성장세가 계속될 것이...,2022.5.31 10:18,한국경제,https://finance.naver.com/item/news_read.naver...


In [6]:
import pickle

with open('words.pickle', 'rb') as f:
    words = pickle.load(f)
    
with open('ranks.pickle', 'rb') as f:
    ranks = pickle.load(f)

In [7]:
print(len(naver_article), len(words), len(ranks))

3089 3089 3089


In [8]:
for i in range(len(naver_article)):
    article_date = naver_article.iloc[i]['Date'].split(' ')[0].split('.')
    article_date = " ".join(article_date)
    
    try:
        stock_idx = date2idx[article_date]
    except KeyError as e:
        print('없는 날짜: ', article_date)
        continue
    
    stock_date = naver_stock.iloc[stock_idx]['Date'].split('.')
    stock_date[1] = stock_date[1].lstrip('0')
    stock_date = " ".join(stock_date)
       
    # 등락률 계산
    diff = naver_stock.iloc[stock_idx]['Close'] - naver_stock.iloc[stock_idx+1]['Close']
    diff /= naver_stock.iloc[stock_idx+1]['Close']
    diff *= 100
    print('{} 등락률: {}'.format(stock_date, diff))
    
    for j in range(len(words[i])):  # 키워드 단어 순회하면서 word_scores 계산
            if words[i][j] in word_scores:  # 이미 계산된 단어의 경우
                word_scores[words[i][j]] += ranks[i][j] * diff
                word_counts[words[i][j]] += 1
            else:  # 처음 계산된 단어의 경우
                word_scores[words[i][j]] = ranks[i][j] * diff
                word_counts[words[i][j]] = 1

2022 5 31 등락률: 2.491103202846975
2022 5 31 등락률: 2.491103202846975
2022 5 31 등락률: 2.491103202846975
2022 5 31 등락률: 2.491103202846975
2022 5 31 등락률: 2.491103202846975
2022 5 31 등락률: 2.491103202846975
2022 5 31 등락률: 2.491103202846975
없는 날짜:  2022 6 1
없는 날짜:  2022 6 1
없는 날짜:  2022 6 1
없는 날짜:  2022 6 1
없는 날짜:  2022 6 1
없는 날짜:  2022 6 1
2022 5 31 등락률: 2.491103202846975
2022 5 31 등락률: 2.491103202846975
2022 5 31 등락률: 2.491103202846975
2022 5 31 등락률: 2.491103202846975
2022 5 30 등락률: 4.074074074074074
2022 5 30 등락률: 4.074074074074074
2022 5 30 등락률: 4.074074074074074
2022 5 30 등락률: 4.074074074074074
2022 5 30 등락률: 4.074074074074074
2022 5 30 등락률: 4.074074074074074
2022 5 30 등락률: 4.074074074074074
2022 5 30 등락률: 4.074074074074074
2022 5 30 등락률: 4.074074074074074
없는 날짜:  2022 5 29
없는 날짜:  2022 5 29
없는 날짜:  2022 5 29
없는 날짜:  2022 5 29
없는 날짜:  2022 5 29
없는 날짜:  2022 5 29
없는 날짜:  2022 5 29
없는 날짜:  2022 5 28
없는 날짜:  2022 5 28
2022 5 27 등락률: 1.5037593984962405
2022 5 27 등락률: 1.5037593984962405
2022 5 2

In [9]:
word_scores

{'네이버': 201.03724394929444,
 '광고': -22.946597312943933,
 '성장': -65.51266635395355,
 '것이': -0.23589710811847553,
 '새벽배송': 5.781942576470753,
 '물류': -14.475179673001453,
 '자격증': 15.94899861573368,
 '발급': 4.030075371777089,
 '투표': 1.8278631572635482,
 '있다': 199.28487792124838,
 '올해': -20.056794552741326,
 '웹툰': -61.635147566220596,
 '1분기': -10.923938473130644,
 '서비스': 69.38317743134502,
 '제공': 3.9396861018938174,
 '대형': 6.181809940102084,
 '5월': 6.054864903313438,
 '대비': -123.19775759967098,
 '외국인': 25.756974068060945,
 '이날': 33.28569232815928,
 '코스피': -3.5199290025452026,
 '환율': 6.854838659052401,
 '순매수': 3.0167956054472094,
 '상승': 56.532272148977476,
 '중국': 9.553584549917874,
 '지난': -16.191879543540026,
 '클로바': 37.68681993550609,
 '케어콜': 47.61344656076626,
 'AI': -126.0730260891916,
 '도입': 0.11545270152439624,
 '지자체': 1.5150989147690184,
 '어르신': 5.924348584128617,
 '솔루션': -8.906662655821453,
 '대화': 3.4276530083280248,
 '주요': 7.5852060077922205,
 '투자': 81.25488620194052,
 '대표': 18.383461

In [10]:
word_counts

{'네이버': 927,
 '광고': 10,
 '성장': 74,
 '것이': 5,
 '새벽배송': 1,
 '물류': 16,
 '자격증': 4,
 '발급': 4,
 '투표': 1,
 '있다': 265,
 '올해': 16,
 '웹툰': 50,
 '1분기': 9,
 '서비스': 168,
 '제공': 13,
 '대형': 1,
 '5월': 1,
 '대비': 45,
 '외국인': 11,
 '이날': 8,
 '코스피': 5,
 '환율': 2,
 '순매수': 6,
 '상승': 15,
 '중국': 2,
 '지난': 21,
 '클로바': 27,
 '케어콜': 16,
 'AI': 147,
 '도입': 2,
 '지자체': 1,
 '어르신': 3,
 '솔루션': 27,
 '대화': 3,
 '주요': 1,
 '투자': 42,
 '대표': 155,
 '출자': 1,
 '지난해': 42,
 '입점': 7,
 '개인': 5,
 '네이버와': 5,
 '코로나19': 8,
 '오른': 4,
 '카카오': 99,
 '날씨': 1,
 '예보': 1,
 '기상청': 1,
 '웨더': 1,
 '위성': 6,
 '영상': 14,
 '쎄트렉아이': 3,
 '클라우드': 21,
 '기반': 3,
 '브랜드': 14,
 '인플루언서': 4,
 '캠페인': 2,
 '커넥트': 1,
 '콘텐츠': 59,
 '브랜드사': 1,
 '미국': 7,
 '시세': 3,
 '증권': 3,
 '실시간': 7,
 '증시': 3,
 '피부': 2,
 '코스알엑스': 1,
 '소비자들': 1,
 '세럼': 1,
 '거래': 9,
 '아루': 1,
 '아바타': 1,
 '센스': 1,
 '식스': 1,
 '웹소설': 7,
 '네이버웹툰': 18,
 '키스': 1,
 '해커톤': 3,
 '베트': 3,
 'HUST': 5,
 '베트남': 3,
 '글로벌': 121,
 '연구': 23,
 'CSAP': 1,
 '인증': 8,
 '클로바노트': 6,
 '버전': 1,
 '일본어': 1,
 '음성': 6,
 '일본': 70,
 '네이버는'

### 등락률 스코어 평균 계산

In [11]:
print(len(word_scores), len(word_counts))

1257 1257


In [12]:
final_scores = {}

for key in word_scores.keys():
    print('{}: {}'.format(key, word_scores[key] / word_counts[key]))
    final_scores[key] = word_scores[key] / word_counts[key]

네이버: 0.2168686558244816
광고: -2.2946597312943933
성장: -0.8853063020804534
것이: -0.04717942162369511
새벽배송: 5.781942576470753
물류: -0.9046987295625908
자격증: 3.98724965393342
발급: 1.0075188429442723
투표: 1.8278631572635482
있다: 0.7520184072499939
올해: -1.253549659546333
웹툰: -1.2327029513244119
1분기: -1.2137709414589604
서비스: 0.4129951037580061
제공: 0.30305277706875516
대형: 6.181809940102084
5월: 6.054864903313438
대비: -2.737727946659355
외국인: 2.3415430970964497
이날: 4.16071154101991
코스피: -0.7039858005090405
환율: 3.4274193295262005
순매수: 0.5027992675745349
상승: 3.768818143265165
중국: 4.776792274958937
지난: -0.7710418830257155
클로바: 1.3958081457594846
케어콜: 2.9758404100478915
AI: -0.8576396332598069
도입: 0.05772635076219812
지자체: 1.5150989147690184
어르신: 1.9747828613762055
솔루션: -0.3298763946600538
대화: 1.1425510027760082
주요: 7.5852060077922205
투자: 1.9346401476652506
대표: 0.11860297921126882
출자: 5.473520613917709
지난해: -0.27296189997016046
입점: 7.083453204026564
개인: 3.4955114851540423
네이버와: 1.2519354813916714
코로나19: 3.871

In [13]:
with open('final_scores.pickle', 'wb') as f:
    pickle.dump(final_scores, f)